In [1]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve
from scipy.constants import mu_0, epsilon_0

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps
    import matrices_assembly as assembly
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


## Physics Constants

In [2]:
K0 = 2 * np.pi * 3E8 * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0

# 2-noded Linear Elements, $P_1$

In [3]:
FINITE_ELEMENT = ("Triangle", 1)

BOUNDARY = [{'tag': 101,'type': 'ABC', 'value': None, 'name': 'x0'},
            {'tag': 102, 'type': 'ABC', 'value': None, 'name': 'xL'}]

MATERIAL = [{'tag': 201, 'name': 'free_space', 'electric_conductivity': 0,
              'relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 202, 'name': 'lossy_slab', 'electric_conductivity': 5E-3,
              'relative_magnetic_permeability': 1, 'relative_electric_permittivity': 2.7}]

INTERFACES = [{'tag': 301, 'type': 'Interface', 'value': None, 'name': 'xa'},
              {'tag': 302, 'type': 'Interface', 'value': None, 'name': 'xb'}]

## FEM Formulation

In [ ]:
# Define o domínio do problema
GEOMETRY = {'a': {'h': WAVELENGTH/20, 'L': WAVELENGTH*2, 'd': WAVELENGTH/4},
            'b': {'h': WAVELENGTH/40, 'L': WAVELENGTH*2, 'd': WAVELENGTH/4},
            'c': {'h': WAVELENGTH/80, 'L': WAVELENGTH*2, 'd': WAVELENGTH/4}}

data_resume = {}
for item in ['a', 'b', 'c']:
    # Criação da malha
    mesh_data = ps.create_p1_domain(
        FINITE_ELEMENT, BOUNDARY, MATERIAL, INTERFACES, GEOMETRY[item], auto_save=False)

    # Montagem das matrizes globais
    Sg, Mg, bg = assembly.global_matrices(mesh_data)

    # Aplicar condições de contorno ABC
    Sg, Mg, bg = assembly.apply_simple_bcs(Sg, Mg, bg, mesh_data)

    # Vetor de potencial global
    ug = spsolve((Sg + Mg).tocsr(), bg.toarray())

    # Dicionário com os resultados
    uh = {key+1: value for key, value in enumerate(ug)}

    # Coeficientes de reflexão e transmissão
    R_coeff = np.abs(uh[1])
    u_inc = np.exp(-1j * K0 * mesh_data['nodes'][4]['xg'][0])
    T_coeff = np.abs(u_inc + uh[4])
    
    # Salvar os resultados
    data_resume[item] = {'R': R_coeff, 'T': T_coeff}

Model lossy_dielectric_slab_abc (1D)
Info     : 7 geometric entities
Info     : 6 Physical Groups
Info     : 42 nodes in total
Info     : 41 1 elements in total
Applying ABC at node 1 with value 6.287535065855045j
Applying ABC at node 4 with value 6.287535065855045j
Model lossy_dielectric_slab_abc (1D)
Info     : 7 geometric entities
Info     : 6 Physical Groups
Info     : 81 nodes in total
Info     : 80 1 elements in total
Applying ABC at node 1 with value 6.287535065855045j
Applying ABC at node 4 with value 6.287535065855045j
Model lossy_dielectric_slab_abc (1D)
Info     : 7 geometric entities
Info     : 6 Physical Groups
Info     : 161 nodes in total
Info     : 160 1 elements in total
Applying ABC at node 1 with value 6.287535065855045j
Applying ABC at node 4 with value 6.287535065855045j


## TABLE 4.14
Magnitudes of Reflection and Transmission Coefficients for Lossy Dielectric Slab  
$\varepsilon_r = 2.7$ and $\sigma = 5 \times 10^{-3} S/m$  
Exact Coefficients: $|R_e| = 0.2434$ and $|T_e| = 0.8384$

In [5]:
exact_values = {'R': 0.2434, 'T': 0.8384}
ps.table_resume(data_resume, exact_values)

,Element Length,|R|,Error in |R| (%),|T|,Error in |T| (%)
0,lambda/20,0.2547,4.65,0.8601,2.59
1,lambda/40,0.2440,0.25,0.8444,0.71
2,lambda/80,0.2413,0.87,0.8400,0.20


Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.